# Метод условного градиента #

#### Выполняла: Кузнецова Мария студентка 311 группы

## Математическое описание метода

Рассмотрим задачу минимизации: 
\begin{equation}
    J(u) \rightarrow inf, u \in U
\end{equation}

Идея метода условного градиента основана на том, что в окрестности точки $u_k$ функционал $J(u)$ можно приближенно представить как
\begin{equation}
J(u) \approx J(u_k) + \langle J'(u_k), u - u_k\rangle
\end{equation}

Так как $J(u_k)$ близко к $J(u)$, то мы стараемся минимизировать скалярное произведение $J_k(u) = \langle J'(u_k), u − u_k\rangle$. Введём обозначение:
\begin{equation}
\bar u_k = argmin_{u \in U} J_k(u)
\end{equation}

Тогда итерационная последовательность рассматриваемого метода описывается как
\begin{equation}
u_{k+1} = u_k + \alpha_k(\bar u_k - u_k),
\end{equation}
где
\begin{equation}
\alpha_k = argmin_{0 \leq \alpha \leq 1} J(u_k + \alpha(\bar u_k - u_k))
\end{equation}

Для обоснования сходимости метода используется теорема:

#### Теорема. #### 
Пусть U - выпуклое, замкнутое, ограниченное множество, $J(u) \in C^1(U)$ и выпукла, $J'(u) \in Lip(U)$ с константой $L > 0, D = diam U$. Тогда
\begin{equation}
J(u_k) - J_{*} \leq \frac{J(u_0) - J_{*}}{1 + \frac{J(u_0) - J_{*}}{2LD}k} = O(\frac{1}{k})
\end{equation}

Если $J(u)$ сильно выпукла, то, кроме этого, выполнено условие
\begin{equation}
\frac{k}{2}||u_k - u_{*}|| \leq J(u_k) - J_{*}
\end{equation}

Источники: Потапов М.М. "Методы оптимизации. Конспект лекций."

## Функция, реализующая метод условного градиента, и её описание

In [30]:
import numpy as np
from scipy.optimize import linprog
from scipy.optimize import bisect

Функция, задающая метод условного градиента

f - функция, которую мы минимизируем, f_k - её производная, A, b - ограничения на x,  eps - точность, k_max - максимальное количество итераций.

In [31]:
def conditional_gradient(f, f_k, f_alpha, A, b, x_0, eps=2e-12, k_max=500):
    for k in range(k_max):
        x = linprog(f_k(x_0), A, b).x                                      # решаем вспомогательную задачу
        x_ = x - x_0                                                       # линейного программирования.
        if np.sign(f_alpha(x_0, x_)(0)) == np.sign(f_alpha(x_0, x_)(1)):
            if f_alpha(x_0, x_)(0) < f_alpha(x_0, x_)(1):
                alpha = 0
            else:                                                          # решем вспомогательную задачу
                alpha = 1                                                  # для нахождения альфа.
        else:                                                              # Используем для этого
            alpha = bisect(f_alpha(x_0, x_), 0, 1)                         # метод деления отрезка пополам
        x_k = x_0 + alpha * x_
        for i in range(5):                                                # проверка, что новое x не выходит за границы
            if x_k[i] > b[i]:
                x_k[i] = b[i + 5]
        if abs(f(x_k) - f(x_0)) < eps:
            return x_k, k
        x_0 = x_k
    return x_0, k

Используя метод linprog, решаем вспомогательную задачу линейного программирования и находим $\bar u_k$. Далее используя метод деления отрезка пополам bisect и находим $\alpha_k$. После находим $u_{k+1}$ и переходим к следующей итерации.

## Решение задачи оптимизации

Функция, которую мы минимизируем

\begin{equation}
J(x) = \sum_{i=1}^{5} [(ln(x_i - 2))^2 + (ln(10 - x_i))^2 - (\prod_{i=1}^{5}x_i)^2] \rightarrow min
\end{equation}
\begin{equation}
2.001 \leq x_i \leq 9.999, i = 1, ..., 5
\end{equation}

In [32]:
def f(x):
    lg_1 = (np.log(x - 2))**2
    lg_2 = (np.log(10 - x))**2
    mul = np.prod(x)**2
    
    return (lg_1 + lg_2 - mul).sum()

Градиент функции, которую мы минимизируем

In [33]:
def f_k(x):
    x_ = np.zeros(5)
    for i in range(5):
        x_[i] += 2 * np.log(x[i] - 2) / (x[i] - 2)
        x_[i] -= 2 * np.log(10 - x[i]) / (10 - x[i])
        x_[i] -= 2 * 5 * np.prod(x)**2 / x[i]
    return x_

Производная по alpha функции f(x_0 + alpha * x_)

In [34]:
def f_alpha(x_0, x_):
    def temp(alpha):
        res = 0
        for i in range(5):
            res += 2 * x_[i] * np.log(x_0[i] + alpha * x_[i] - 2) / (x_0[i] + alpha * x_[i] - 2)
            res -= 2 * x_[i] * np.log(10 - x_0[i] - alpha * x_[i]) / (10 - x_0[i] - alpha * x_[i])
            res -= 2 * 5 * np.prod(x_0 + alpha * x_) * x_[i] * np.prod(np.delete((x_0 + alpha * x_), i))
        return res
    return temp

Задаём ограничения

In [35]:
A = np.array([[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [-1, 0, 0, 0, 0], 
              [0, -1, 0, 0, 0],
              [0, 0, -1, 0, 0],
              [0, 0, 0, -1, 0],
              [0, 0, 0, 0, -1]])     
b = np.array([9.999, 9.999, 9.999, 9.999, 9.999, 2.001, 2.001, 2.001, 2.001, 2.001])   

Задаём все ограничения, точность, начальную точку и решаем нашу задачу

In [36]:
x_0 = np.array([3, 9.99, 9.999, 8, 9.999])                             # начальная точка
eps = 2e-12                                                         # точность до которой мы вычисляем минимум
k_max = 1000                                                        # количество итераций

x, k = conditional_gradient(f, f_k, f_alpha, A, b, x_0, eps, k_max) # вычисление минимума методом условного градиента

print('x_min =', x)     
print('number of iterations =', k + 1)
print('Answer f(x_min) =', f(x))                                    # вывод ответа

x_min = [9.999 9.999 9.999 9.999 9.999]
number of iterations = 2
Answer f(x_min) = -49950022233.797874


f - функция, которую мы минимизируем, f_k - её производная, A, b - ограничения на x,  eps - точность, k_max - максимальное количество итераций.

## Таблица решений в зависимости от выбора начальной точки

Запускаем наш метод из разных начальных приближений и записываем результаты

In [10]:
table = []
for x in np.arange(2.001, 10.0, 0.001):
    x = round(x, 3)
    x_0 = np.array([x, x, x, x, x])
    x_min, k = conditional_gradient(f, f_k, f_alpha, A, b, x_0, eps, k_max)
    f_min = f(x_min)
    table.append(['$x_i$ = ' + str(x_0[0]), str(k + 1), '$x_i$ = ' + str(x_min[0]), str(f_min)])

In [13]:
for x in table:
    if x[1] != '2':
        print(x)

['$x_i$ = 9.999', '1', '$x_i$ = 9.999', '-49950022233.797874']


Сохраним результаты в таблицы, чтобы в дальнейшим можно было удобно сформировать отчёт

In [11]:
import csv

with open('table.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['x_0', 'iterations', 'x_{min}', 'f(x_{min})'])
    for row in table:
        writer.writerow(row)

with open('table.csv') as f:
    print(f.read())

x_0,iterations,x_{min},f(x_{min})

$x_i$ = 2.001,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.002,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.003,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.004,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.005,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.006,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.007,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.008,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.009,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.01,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.011,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.012,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.013,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.014,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.015,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.016,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.017,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.018,2,$x_i$ = 9.999,-49950022233.797874

$x_i$ = 2.019,2,$x_i$ = 9.999,-49950022233.79787

In [28]:
import csv

with open('table_1.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['x_0', 'iterations', 'x_{min}', 'f(x_{min})'])
    for row in table[:10]:
        writer.writerow(row)
    writer.writerow(['$\ldots$', '$\ldots$', '$\ldots$', '$\ldots$'])
    for row in table[-10:]:
        writer.writerow(row)